## Saving as Partitioned Tables

We can also create partitioned tables while using `saveAsTable` function to write data from Dataframe into a metastore table.

* Let us create partitioned table for `orders` by `order_month`.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Spark Metastore'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
spark.conf.set('spark.sql.shuffle.partitions', '2')

### Tasks

Let us perform tasks related to partitioned tables.
* Read data from file into data frame.
* Add additional column which will be used to partition the data.
* Use `saveAsTable` to write the data in the Dataframe to a new table in the database we are attached to. The folder related to the table will be created using default location.

In [3]:
import getpass
username = getpass.getuser()

In [4]:
spark.sql(f'CREATE DATABASE IF NOT EXISTS {username}_retail')

""


In [5]:
spark.catalog.setCurrentDatabase(f'{username}_retail')

In [6]:
spark.catalog.currentDatabase()

'itversity_retail'

In [7]:
orders_path = '/public/retail_db/orders'

In [8]:
%%sh

hdfs dfs -ls /public/retail_db/orders

Found 1 items
-rw-r--r--   2 hdfs supergroup    2999944 2021-01-28 09:27 /public/retail_db/orders/part-00000


In [9]:
spark.sql('DROP TABLE IF EXISTS orders_part')

""


In [10]:
from pyspark.sql.functions import date_format

In [11]:
orders = spark. \
    read. \
    csv(orders_path,
        schema='''order_id INT, order_date DATE,
                  order_customer_id INT, order_status STRING
               '''
       ). \
    withColumn('order_month', date_format('order_date', 'yyyyMM'))

In [12]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- order_customer_id: integer (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_month: string (nullable = true)



In [13]:
orders.write.saveAsTable?

Signature:
orders.write.saveAsTable(
    name,
    format=None,
    mode=None,
    partitionBy=None,
    **options,
)
Docstring:
Saves the content of the :class:`DataFrame` as the specified table.

In the case the table already exists, behavior of this function depends on the
save mode, specified by the `mode` function (default to throwing an exception).
When `mode` is `Overwrite`, the schema of the :class:`DataFrame` does not need to be
the same as that of the existing table.

* `append`: Append contents of this :class:`DataFrame` to existing data.
* `overwrite`: Overwrite existing data.
* `error` or `errorifexists`: Throw an exception if data already exists.
* `ignore`: Silently ignore this operation if data already exists.

:param name: the table name
:param format: the format used to save
:param mode: one of `append`, `overwrite`, `error`, `errorifexists`, `ignore`                      (default: error)
:param partitionBy: names of partitioning columns
:param options: all other stri

In [14]:
orders. \
    write. \
    saveAsTable(
        'orders_part',
        mode='overwrite',
        partitionBy='order_month'
    )

In [15]:
%%sh

hdfs dfs -ls -R /user/`whoami`/warehouse/`whoami`_retail.db/orders_part

-rw-r--r--   3 itversity itversity          0 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/_SUCCESS
drwxr-xr-x   - itversity itversity          0 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307
-rw-r--r--   3 itversity itversity      14435 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201307/part-00000-8bd761cc-cdd4-4090-97e1-48adb5297ea8.c000.snappy.parquet
drwxr-xr-x   - itversity itversity          0 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308
-rw-r--r--   3 itversity itversity      49997 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201308/part-00000-8bd761cc-cdd4-4090-97e1-48adb5297ea8.c000.snappy.parquet
drwxr-xr-x   - itversity itversity          0 2021-03-13 15:49 /user/itversity/warehouse/itversity_retail.db/orders_part/order_month=201309
-rw-r--r--   3 itversity itversity

In [16]:
spark. \
    read. \
    parquet(f'/user/{username}/warehouse/{username}_retail.db/orders_part/order_month=201308'). \
    show()

+--------+----------+-----------------+---------------+
|order_id|order_date|order_customer_id|   order_status|
+--------+----------+-----------------+---------------+
|    1297|2013-08-01|            11607|       COMPLETE|
|    1298|2013-08-01|             5105|         CLOSED|
|    1299|2013-08-01|             7802|       COMPLETE|
|    1300|2013-08-01|              553|PENDING_PAYMENT|
|    1301|2013-08-01|             1604|PENDING_PAYMENT|
|    1302|2013-08-01|             1695|       COMPLETE|
|    1303|2013-08-01|             7018|     PROCESSING|
|    1304|2013-08-01|             2059|       COMPLETE|
|    1305|2013-08-01|             3844|       COMPLETE|
|    1306|2013-08-01|            11672|PENDING_PAYMENT|
|    1307|2013-08-01|             4474|       COMPLETE|
|    1308|2013-08-01|            11645|        PENDING|
|    1309|2013-08-01|             2367|         CLOSED|
|    1310|2013-08-01|             5602|        PENDING|
|    1311|2013-08-01|             5396|PENDING_P

In [17]:
spark. \
    read. \
    parquet(f'/user/{username}/warehouse/{username}_retail.db/orders_part'). \
    show()

+--------+----------+-----------------+---------------+-----------+
|order_id|order_date|order_customer_id|   order_status|order_month|
+--------+----------+-----------------+---------------+-----------+
|   15488|2013-11-01|             8987|PENDING_PAYMENT|     201311|
|   15489|2013-11-01|             5359|PENDING_PAYMENT|     201311|
|   15490|2013-11-01|            10149|       COMPLETE|     201311|
|   15491|2013-11-01|            10635|        ON_HOLD|     201311|
|   15492|2013-11-01|             7784|PENDING_PAYMENT|     201311|
|   15493|2013-11-01|             1104|        ON_HOLD|     201311|
|   15494|2013-11-01|             7313|     PROCESSING|     201311|
|   15495|2013-11-01|             7067|         CLOSED|     201311|
|   15496|2013-11-01|            12153|PENDING_PAYMENT|     201311|
|   15497|2013-11-01|            11115|PENDING_PAYMENT|     201311|
|   15498|2013-11-01|            11195|       COMPLETE|     201311|
|   15499|2013-11-01|             7113|         

In [18]:
spark.sql('SHOW PARTITIONS orders_part').show()

+------------------+
|         partition|
+------------------+
|order_month=201307|
|order_month=201308|
|order_month=201309|
|order_month=201310|
|order_month=201311|
|order_month=201312|
|order_month=201401|
|order_month=201402|
|order_month=201403|
|order_month=201404|
|order_month=201405|
|order_month=201406|
|order_month=201407|
+------------------+



In [19]:
spark.read.table('orders_part').show()

+--------+----------+-----------------+---------------+-----------+
|order_id|order_date|order_customer_id|   order_status|order_month|
+--------+----------+-----------------+---------------+-----------+
|   15488|2013-11-01|             8987|PENDING_PAYMENT|     201311|
|   15489|2013-11-01|             5359|PENDING_PAYMENT|     201311|
|   15490|2013-11-01|            10149|       COMPLETE|     201311|
|   15491|2013-11-01|            10635|        ON_HOLD|     201311|
|   15492|2013-11-01|             7784|PENDING_PAYMENT|     201311|
|   15493|2013-11-01|             1104|        ON_HOLD|     201311|
|   15494|2013-11-01|             7313|     PROCESSING|     201311|
|   15495|2013-11-01|             7067|         CLOSED|     201311|
|   15496|2013-11-01|            12153|PENDING_PAYMENT|     201311|
|   15497|2013-11-01|            11115|PENDING_PAYMENT|     201311|
|   15498|2013-11-01|            11195|       COMPLETE|     201311|
|   15499|2013-11-01|             7113|         

In [21]:
spark.sql('SELECT order_month, count(1) FROM orders_part GROUP BY order_month').show()

+-----------+--------+
|order_month|count(1)|
+-----------+--------+
|     201406|    5308|
|     201403|    5778|
|     201308|    5680|
|     201404|    5657|
|     201311|    6381|
|     201401|    5908|
|     201312|    5892|
|     201309|    5841|
|     201402|    5635|
|     201405|    5467|
|     201310|    5335|
|     201407|    4468|
|     201307|    1533|
+-----------+--------+



In [22]:
spark.read.table('orders_part'). \
    groupBy('order_month'). \
    count(). \
    show()

+-----------+-----+
|order_month|count|
+-----------+-----+
|     201406| 5308|
|     201403| 5778|
|     201308| 5680|
|     201404| 5657|
|     201311| 6381|
|     201401| 5908|
|     201312| 5892|
|     201309| 5841|
|     201402| 5635|
|     201405| 5467|
|     201310| 5335|
|     201407| 4468|
|     201307| 1533|
+-----------+-----+

